Import Libraries

In [1]:
# !pip install contractions

In [2]:
import pandas as pd
import numpy as np
import string
import nltk
import contractions
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Load Data

In [3]:
input_data = pd.read_csv("C:\VOLD\ML\Dphi\Train_Data.csv")
test_data = pd.read_csv("C:\VOLD\ML\Dphi\Test_Data.csv")
sample_submission = pd.read_csv("C:\VOLD\ML\Dphi\Sample_Submission.csv")

In [4]:
X_train, X_val = train_test_split(input_data,test_size=0.2,random_state=42)

In [5]:
X_train.shape

(5988, 17)

In [6]:
X_val.shape

(1498, 17)

In [7]:
X_train.head(3)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,average_review_rating
426,c8ec89c34a411f693e6c111fcd0920c2,Forces of Valor Diecast Model - German Kubelwa...,Forces Of Valor,£34.32,15 new,1,1.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,NaN,Forces of Valor 1/32 WWII German Kubelwagen 19...,Technical Details Item Weight304 g Product Dim...,Forces of Valor 1/32 WWII German Kubelwagen 19...,http://www.amazon.co.uk/Revell-Scale-Junkers-A...,NaN,Four Stars // 4.0 // 6 Aug. 2014 // By\n \n...,NaN,4.0
1595,7c005700a861a98e3a27a123c87266ff,Star Wars Party - Darth Vader 18 inch Foil Bal...,Star Wars,£1.88,4 new,4,2.0,Characters & Brands > Star Wars > Toys,http://www.amazon.co.uk/Amscan-25-inch-Helmet-...,NaN,Technical Details Item Weight23 g Product Dime...,NaN,NaN,NaN,Dearth Vader baloon // 5.0 // 11 Nov. 2015 // ...,"{""seller""=>[{""Seller_name_1""=>""BIRTHDAY BUBBLE...",4.0
805,2da7f3bd2a5ccf5b34e91e3aa998224e,Pink Butterfly Garland,Truly Madly Deeply,£10.00,NaN,1,2.0,"Party Supplies > Banners, Stickers & Confetti ...",NaN,This very pretty Pink Butterfly Garland has bu...,Technical Details Manufacturer recommended age...,This very pretty Pink Butterfly Garland has bu...,NaN,NaN,Five Stars // 5.0 // 23 Jan. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""Truly Madly Deep...",5.0


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5988 entries, 426 to 7270
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      5988 non-null   object 
 1   product_name                                 5988 non-null   object 
 2   manufacturer                                 5984 non-null   object 
 3   price                                        5120 non-null   object 
 4   number_available_in_stock                    4438 non-null   object 
 5   number_of_reviews                            5988 non-null   object 
 6   number_of_answered_questions                 5524 non-null   float64
 7   amazon_category_and_sub_category             5602 non-null   object 
 8   customers_who_bought_this_item_also_bought   5321 non-null   object 
 9   description                                  5605 non-null   object 
 10

In [9]:
X_train.describe(include='object')

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
count,5988,5988,5984,5120,4438,5988,5602,5321,5605,5954,5605,4121,500,5985,4124
unique,5988,5975,1976,1888,80,149,223,5221,5181,5953,5181,4023,498,5954,3957
top,6352e4a80cc4a10b61e61a38353d007c,Barbie Advent Calendar,Oxford Diecast,£9.99,2 new,1,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,http://www.amazon.co.uk/Scalextric | http://ww...,"Welcome to k2, we offer combined items postage...",Technical Details Item Weight680 g Product Dim...,"Welcome to k2, we offer combined items postage...",http://www.amazon.co.uk/Scalextric | http://ww...,does it have an inflated bottom? // No,Five Stars // 5.0 // 14 Jan. 2016 // By\n \...,"{""seller""=>{""Seller_name_1""=>""k2-accessories"",..."
freq,1,2,109,95,807,2629,544,34,73,2,73,21,2,3,16


Product information can be appended to review to add more meaning to the review and increase the relation between product and review. Adding rest of the object info might not be useful as there is a higher number of missing values or because of the wide range of unique values as is the case with column manufacturer.

In [10]:
X_train.describe()

,number_of_answered_questions,average_review_rating
count,5524.000000,5988.000000
mean,1.817705,4.707315
std,2.505234,0.372519
min,1.000000,3.000000
25%,1.000000,4.500000
50%,1.000000,5.000000
75%,2.000000,5.000000
max,39.000000,5.000000


Feature Processing

In [11]:
X_train['customer_reviews'].iloc[0]

'Four Stars // 4.0 // 6 Aug. 2014 // By\n    \n    Mr Michael James Bardin\n  \n on 6 Aug. 2014 // very nice item'

In [12]:
X_train['description'].iloc[0]

'Forces of Valor 1/32 WWII German Kubelwagen 1944 # 82007'

The review has rating given along with each review. This can be separated out to a new column and removed from review as the date and the person who submitted the review does not provide much useful information to the review. We can add the description along with the review during this step.

In [13]:
def add_review_details(data):
    data = data.dropna(subset=['customer_reviews','description'])
    data_with_rating_details = data.copy()
    data_with_rating_details['rating_list'] = np.empty((len(data),0)).tolist()
    data_with_rating_details['enrich_review'] = data['customer_reviews']
    for index,row in data.iterrows():
        review_text = row['customer_reviews']
        description = row['description']
        rating_list = []

        count = 0
        start_index = 0
        start = False
        new_review = row['customer_reviews']
        for i in range(len(review_text)):
            if review_text[i:i+2] == "//":
                if start == True:
                    count += 1
                if review_text[i+3].isdigit() and review_text[i+5].isdigit() and review_text[i+4] == ".":
                    count = 1
                    start = True
                    start_index = i
                    rating_list.append(float(review_text[i+3:i+6]))
                    # print(review_text[i+3:i+6])
            if count == 4 and start:
                start = False
                new_review = new_review.replace(review_text[start_index-1:i+2],".")
        # print(index,new_review,"\n\n")
        data_with_rating_details.at[index,'enrich_review'] = description + new_review
        data_with_rating_details.at[index,'rating_list'] = rating_list
    
    data_with_rating_details['char_count'] = data_with_rating_details['enrich_review'].apply(len)
    data_with_rating_details['word_count'] = data_with_rating_details['enrich_review'].apply(lambda x: len(x.split()))
    data_with_rating_details['word_density'] = data_with_rating_details['char_count'] / data_with_rating_details['word_count']
    data_with_rating_details['punctuation_count'] = data_with_rating_details['enrich_review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
    data_with_rating_details['title_word_count'] = data_with_rating_details['enrich_review'].apply(lambda x: len([word for word in x.split() if word.istitle()]))
    data_with_rating_details['upper_case_word_count'] = data_with_rating_details['enrich_review'].apply(lambda x:len([word for word in x.split() if word.isupper()]))

    return data_with_rating_details

In [14]:
X_train_with_rating_details = add_review_details(X_train)
X_val_with_rating_details = add_review_details(X_val)
X_train_with_rating_details[['customer_reviews','enrich_review','rating_list']].head(6)

,customer_reviews,enrich_review,rating_list
426,Four Stars // 4.0 // 6 Aug. 2014 // By\n \n...,Forces of Valor 1/32 WWII German Kubelwagen 19...,[4.0]
805,Five Stars // 5.0 // 23 Jan. 2015 // By\n \...,This very pretty Pink Butterfly Garland has bu...,[5.0]
410,Classic Salty // 5.0 // 26 Jan. 2016 // By\n ...,Thomas and Friends Minis they are a collection...,[5.0]
6260,Five little men // 5.0 // 25 Nov. 2011 // By\n...,Product Description “Five little men in a flyi...,"[5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 3.0, 5.0]"
6043,sleek // 5.0 // 2 Dec. 2013 // By\n \n S...,Product Description These are the best poker c...,"[5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0]"
438,"""Great Gardener"" // 4.0 // 27 Oct. 2014 // By\...","Castle In The Sky Robot Soldier Action Figure""...",[4.0]


Some of the columns which represent numeric values have object datatype. This can be converted using a suitable approach.

In [15]:
def clean_numeric_values(data):
    data['number_of_reviews'] = data['number_of_reviews'].str.replace(',','').astype(int)
    return data
# X_train[(X_train['price'].isnull() == False) & (X_train['price'].str.contains("-"))]['price']
# X_train['number_available_in_stock'].str.replace('[^a-zA-Z]','',regex=True).unique()

Column 'price' has continous values, therefore we need to use an averaging technique or drop these values for converting to numeric values. Also the 'number_available_in_stock' has different furnishing status along with the numeric value. This can be separated out to a new column and one hot encoding can be applied as the number of unique values are low. Skipping both these for now as these can be tested later to see if it improves the model performance.

In [16]:
X_train_enriched = clean_numeric_values(X_train_with_rating_details)
X_val_enriched = clean_numeric_values(X_val_with_rating_details)

In [17]:
X_train_enriched.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5602 entries, 426 to 7270
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      5602 non-null   object 
 1   product_name                                 5602 non-null   object 
 2   manufacturer                                 5600 non-null   object 
 3   price                                        4803 non-null   object 
 4   number_available_in_stock                    4209 non-null   object 
 5   number_of_reviews                            5602 non-null   int32  
 6   number_of_answered_questions                 5184 non-null   float64
 7   amazon_category_and_sub_category             5229 non-null   object 
 8   customers_who_bought_this_item_also_bought   5014 non-null   object 
 9   description                                  5602 non-null   object 
 10

In [18]:
numeric_columns = X_train_enriched.select_dtypes('number').columns.to_list()
numeric_columns.remove("number_of_answered_questions")
numeric_columns.remove('average_review_rating')

We will now move forward to text processing for reviews

In [19]:
stop_words = nltk.corpus.stopwords.words('english')

ps = nltk.porter.PorterStemmer()

def text_processor(input_text):
    input_text = str(input_text.lower())
    
    input_text = contractions.fix(input_text)

    input_text = re.sub(r'[^a-zA-Z]',r' ', input_text)
    input_text = re.sub(r'nbsp',r'',input_text)

    input_text = " ".join([ps.stem(word) for word in input_text.split()])

    return input_text

stp = np.vectorize(text_processor)

In [20]:
X_train_enriched['clean_review'] = stp(X_train_enriched['enrich_review'].values)
X_val_enriched['clean_review'] = stp(X_val_enriched['enrich_review'])

In [21]:
X_train_enriched.head(2)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,...,average_review_rating,rating_list,enrich_review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,clean_review
426,c8ec89c34a411f693e6c111fcd0920c2,Forces of Valor Diecast Model - German Kubelwa...,Forces Of Valor,£34.32,15 new,1,1.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,NaN,Forces of Valor 1/32 WWII German Kubelwagen 19...,...,4.0,[4.0],Forces of Valor 1/32 WWII German Kubelwagen 19...,82,14,5.857143,3,6,1,forc of valor wwii german kubelwagen four star...
805,2da7f3bd2a5ccf5b34e91e3aa998224e,Pink Butterfly Garland,Truly Madly Deeply,£10.00,NaN,1,2.0,"Party Supplies > Banners, Stickers & Confetti ...",NaN,This very pretty Pink Butterfly Garland has bu...,...,5.0,[5.0],This very pretty Pink Butterfly Garland has bu...,919,153,6.006536,18,27,0,thi veri pretti pink butterfli garland ha butt...


In [22]:
X_train_final = X_train_enriched.reset_index(drop=True)
X_val_final = X_val_enriched.reset_index(drop=True)
y_train = X_train_final['average_review_rating']
y_val = X_val_final['average_review_rating']
X_train_final = X_train_final[numeric_columns]
X_val_final = X_val_final[numeric_columns]

In [23]:
cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,1))
X_train_cv = cv.fit_transform(X_train_enriched['clean_review']).toarray()
X_train_cv = pd.DataFrame(X_train_cv,columns=cv.get_feature_names())

X_val_cv = cv.transform(X_val_enriched['clean_review']).toarray()
X_val_cv = pd.DataFrame(X_val_cv,columns=cv.get_feature_names())

X_train_cv.head()

,aa,aaa,aaaa,aaaaarghmaz,aaaah,aaaarrrrrr,aaahhh,aaar,aaf,aaghhhhh,...,zvezda,zwar,zwart,zwei,zx,zxxxx,zygon,zymak,zyudenshi,zzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X_train_comb = pd.concat([X_train_final,X_train_cv],axis=1)
X_val_comb = pd.concat([X_val_final,X_val_cv],axis=1)

X_train_comb.head(2)

,number_of_reviews,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,aa,aaa,aaaa,...,zvezda,zwar,zwart,zwei,zx,zxxxx,zygon,zymak,zyudenshi,zzzzz
0,1,82,14,5.857143,3,6,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,919,153,6.006536,18,27,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Model Training and Evaluation

We cannot use logistic regression as the values here are continous

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 

lr = LinearRegression()

In [26]:
lr.fit(X_train_comb,y_train)

LinearRegression()

In [27]:
linear_val_predictions = lr.predict(X_val_comb)

In [28]:
mean_squared_error(y_val,linear_val_predictions)

2.604413959115211

In [29]:
linear_val_predictions.sort()

In [30]:
linear_val_predictions[:10]

array([-31.96546883,  -8.87229384,  -2.16144806,  -2.08675037,
        -1.84998077,  -1.60731929,  -1.50382849,  -1.47330077,
        -0.74995048,  -0.70820038])

In [31]:
linear_val_predictions[-10:]

array([ 8.72418159,  9.15039072,  9.3744083 ,  9.64445465,  9.68199455,
        9.92288622,  9.99711796, 10.10760225, 10.72414579, 11.7830819 ])

For linear model, we are getting values above 5 and below 0. Also the MSE value is too high which denotes the model is not performing well. The model seems to be too simple to capture all the complex features. So, let's try a more complex model

In [32]:
from sklearn.ensemble import RandomForestRegressor

In [33]:
rfr = RandomForestRegressor()

In [34]:
rfr.fit(X_train_comb,y_train)

RandomForestRegressor()

In [35]:
rfr_val_predictions = rfr.predict(X_val_comb)

In [36]:
rfr_val_predictions.sort()

In [37]:
rfr_val_predictions[:10]

array([4.   , 4.003, 4.01 , 4.012, 4.022, 4.025, 4.03 , 4.03 , 4.035,
       4.093])

In [38]:
rfr_val_predictions[-10:]

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [39]:
rfr_train_predictions = rfr.predict(X_train_comb)

In [40]:
mean_squared_error(y_val,rfr_val_predictions)

0.1912029595170455

In [41]:
mean_squared_error(y_train,rfr_train_predictions)

0.013408478936094273

Random Forest model seems to be doing a good job. There is also a good similarity between the train data and validation data results which denotes the absence of overfitting in the model.

Predict Results for Test Data

In [42]:
X_test_enriched = add_review_details(test_data)
# Not doing integer conversion as number of reviews is already an integer value
# X_test_enriched = clean_numeric_values(X_test_enriched) 
X_test_enriched['clean_review'] = stp(X_test_enriched['enrich_review'])

In [43]:
X_test_final = X_test_enriched.reset_index(drop=False)

In [44]:
X_test_cv = cv.transform(X_test_enriched['clean_review']).toarray()
X_test_cv = pd.DataFrame(X_test_cv,columns=cv.get_feature_names())

X_test_comb = pd.concat([X_test_final[numeric_columns],X_test_cv],axis=1)

In [45]:
rfr_test_predictions = rfr.predict(X_test_comb)

In [46]:
rfr_test_predictions

array([4.99 , 4.419, 4.405, ..., 5.   , 4.793, 4.827])

In [47]:
output_df = pd.DataFrame(rfr_test_predictions,columns=['prediction'])
output_df.index = X_test_final['index']

In [48]:
test_data.index.to_list()[:15]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [49]:
output_df.head(15)

,prediction
index,
0,4.990
1,4.419
2,4.405
3,4.683
4,4.975
5,4.407
6,4.520
7,4.784
8,4.590


In [53]:
result = output_df.join(test_data['uniq_id'],how="right")
result.drop('uniq_id',axis=1,inplace=True)

Fill up the NA values with 0 to successfully submit the predictions.

In [54]:
result.fillna(0,inplace=True)

In [55]:
result.to_csv("C://VOLD/ML/Dphi/prediction_results.csv",index=False)